# Домашнее задание 1 $-$ 10 баллов
Барабанщиков Лев Романович

1. Загрузите набор данных lenta-ru-news с помощью библиотеки Corus для задачи классификации текстов по топикам (пригодятся атрибуты title, text, topic)- 1 балл

In [1]:
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

--2025-09-23 18:15:28--  https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/87156914/0b363e00-0126-11e9-9e3c-e8c235463bd6?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-09-23T15%3A53%3A25Z&rscd=attachment%3B+filename%3Dlenta-ru-news.csv.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-09-23T14%3A53%3A02Z&ske=2025-09-23T15%3A53%3A25Z&sks=b&skv=2018-11-09&sig=rZEpKrZdMPo0xNionYaYppI4my4BoDE%2F8w6lJDHEriw%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1ODY0MDgyOCwibmJmIjoxNzU4NjQwNTI4LCJwYXRoIjoicmVsZWFzZWFzc2V0

In [1]:
from corus import load_lenta

path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

In [2]:
type(records)

generator

In [3]:
from tqdm import tqdm

data = []

for record in tqdm(records):
    if record.topic is None:
        continue
    data.append({
        'title': record.title,
        'text': record.text,
        'topic': record.topic
    })

0it [00:00, ?it/s]

739351it [00:13, 56645.11it/s]


In [4]:
import pandas as pd

df = pd.DataFrame(data)

df

,title,text,topic
0,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия
1,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт
2,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия
3,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир
4,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир
...,...,...,...
739346,Южно-Сахалинск объявлен очагом холеры,Сегодня областной центр Сахалина и Курил получ...,Россия
739347,Леворадикалы создают предвыборный блок,Бывший шеф Службы безопасности президента Але...,Россия
739348,В горах Дагестана идут активные боевые действия,Сегодня утром в районах дагестанских селений Ч...,Россия
739349,Карачаево-Черкесия раскололась по национальном...,Намеченная на сегодняшний день церемония вступ...,Россия


In [5]:
df.topic.value_counts()

topic
Россия               160519
Мир                  136680
Экономика             79538
Спорт                 64421
Культура              53803
Бывший СССР           53402
Наука и техника       53136
Интернет и СМИ        44675
Из жизни              27611
Дом                   21734
Силовые структуры     19596
Ценности               7766
Бизнес                 7399
Путешествия            6408
69-я параллель         1268
Крым                    666
Культпросвет            340
                        203
Легпром                 114
Библиотека               65
Оружие                    3
ЧМ-2014                   2
МедНовости                1
Сочи                      1
Name: count, dtype: int64

2. Подготовьте данные к обучению: - 3 балла

Предобработайте данные: реализуйте оптимальную, на ваш взгляд, предобработку текстов (нормализация, очистка, стемминг/лемматизация и т.п.) и таргета.

Кратко опишите пайплайн, на котором остановились, и почему.

Разделите датасет на обучающую, валидационную и тестовую выборки со стратификацией в пропорции 60/20/20. В качестве целевой переменной используйте атрибут `topic`

In [6]:
import nltk
import pymorphy2
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')

morph = pymorphy2.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package punkt to /Users/bitcoin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/bitcoin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/bitcoin/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
import re

def preprocess(text):
    # Очистка
    text = re.sub(r'[^а-яёa-z\s]|\d+', '', text.lower())
    tokens = word_tokenize(text, language = "russian")
    # Лемматизация
    tokens = [morph.parse(t)[0].normal_form for t in tokens if t not in stop_words and len(t) > 2]
    return ' '.join(tokens)

In [9]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, nb_workers=10)


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [10]:
df['content'] = df['title'] + ' ' + df['text']

df['processed_content'] = df['content'].parallel_apply(preprocess)

In [12]:
# Очистка памяти после обработки данных
import gc

# Удаляем исходные данные для освобождения памяти
del data
del records

# Удаляем промежуточную колонку content, если она больше не нужна
if 'content' in df.columns:
    df.drop('content', axis=1, inplace=True)

# Принудительный запуск сборщика мусора
gc.collect()

print("Память очищена")

# Проверяем использование памяти
import psutil
import os

process = psutil.Process(os.getpid())
memory_mb = process.memory_info().rss / 1024 / 1024
print(f"Использование памяти процессом: {memory_mb:.2f} MB")


Память очищена
Использование памяти процессом: 7625.25 MB


**Описание пайплайна:** Текст из колонок title и text объединяется в единую колонку content. Далее каждый текст проходит через несколько этапов предобработки:
1. приведение к нижнему регистру;
2. фильтрация символов, не являющихся буквами или пробелами;
3. токенизация, удаление стоп-слов и коротких токенов
4. лемматизация $-$ приведение слов к нормальной форме.

Обработанные тексты сохраняются в новой колонке processed_content.
Для распараллеливания процесса используется библиотека pandarallel с прогресс-баром

Разделение датасета

In [13]:
from sklearn.model_selection import train_test_split

X = df['processed_content']
y = df['topic']

X_train, X_other, y_train, y_other = train_test_split(X, y, test_size=0.4, random_state=777)
X_valid, X_test, y_valid, y_test = train_test_split(X_other, y_other, test_size=0.5, random_state=777)

In [14]:
print(y_train.shape, y_valid.shape, y_test.shape)

(443610,) (147870,) (147871,)


3. Замерьте базовое качество с любым dummy-бейзлайном $-$ 0.5 балла

In [15]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

dummy = DummyClassifier(strategy='most_frequent', random_state=777)
dummy.fit(X_train, y_train)
y_valid_pred_dummy = dummy.predict(X_valid)
print(f'Точность на бейзлайне: {accuracy_score(y_valid, y_valid_pred_dummy):.4f}')

Точность на бейзлайне: 0.2168


4. Обучите модель sklearn.linear_model.LogisticRegression с двумя вариантами векторизации: 2 балла
- sklearn.feature_extraction.text.CountVectorizer
- sklearn.feature_extraction.text.TfidfVectorizer

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# TF-IDF

tfidf = Pipeline([
    ('vectorize', TfidfVectorizer(max_features=10000)),
    ('classify', LogisticRegression(max_iter=1000, random_state=777))
])

tfidf.fit(X_train, y_train)

y_pred_tfidf = tfidf.predict(X_valid)

print(f'TF-IDF + LogReg accuracy: {accuracy_score(y_valid, y_pred_tfidf):.4f}')

TF-IDF + LogReg accuracy: 0.8353


In [17]:
# CountVectorizer
count = Pipeline([
    ('vectorize', CountVectorizer(max_features=10000)),
    ('classify', LogisticRegression(max_iter=1000, random_state=777))
])

count.fit(X_train, y_train)

y_pred_count = count.predict(X_valid)

print(f'CountVectorizer accuracy: {accuracy_score(y_valid, y_pred_count):.4f}')


CountVectorizer accuracy: 0.8192


5. Попробуйте улучшить качество, подобрав оптимальные гиперпараметры трансформаций и модели на кросс-валидации 1 балл

Переберем гиперпараметры с помощью GridSearchCV: диапазон n-грамм, количество признаков и параметр регуляризации C. Проведем кросс-валидация на 3 фолдах

In [18]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'vectorize__ngram_range': [(1, 1), (1, 2)],
    'vectorize__max_features': [10000],
    'classify__C': [0.1, 1],
    'classify__max_iter': [100]  # Уменьшим количество итераций для ускорения
}

grid = GridSearchCV(tfidf, param_grid, cv=2, n_jobs=10, verbose=2)  # Запустим на 10 ядрах
grid.fit(X_train, y_train)

print(f'Best configuratiown: {grid.best_params_}')
print(f'Best score: {grid.best_score_:.4f}')

Fitting 2 folds for each of 4 candidates, totalling 8 fits


/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  warnings.warn(
/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max

[CV] END classify__C=0.1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 1); total time= 1.3min
[CV] END classify__C=0.1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 1); total time= 1.3min
[CV] END classify__C=1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 1); total time= 1.3min
[CV] END classify__C=1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 1); total time= 1.3min


/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

[CV] END classify__C=0.1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 2); total time= 2.5min
[CV] END classify__C=0.1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 2); total time= 2.5min
[CV] END classify__C=1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 2); total time= 2.4min
[CV] END classify__C=1, classify__max_iter=100, vectorize__max_features=10000, vectorize__ngram_range=(1, 2); total time= 2.4min
Best configuratiown: {'classify__C': 1, 'classify__max_iter': 100, 'vectorize__max_features': 10000, 'vectorize__ngram_range': (1, 1)}
Best score: 0.8257


/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


6. Оцените качество лучшего пайплайна на отложенной выборке - **0.5 балла**

In [19]:
# Лучшая модель из GridSearchCV
best_model = grid.best_estimator_

# Предсказания на тестовой выборке
y_test_pred = best_model.predict(X_test)

print("Лучшие гиперпараметры:", grid.best_params_)
print(f"Лучший скор на кросс-валидации: {grid.best_score_:.4f}")
6029

Лучшие гиперпараметры: {'classify__C': 1, 'classify__max_iter': 100, 'vectorize__max_features': 10000, 'vectorize__ngram_range': (1, 1)}
Лучший скор на кросс-валидации: 0.8257


6029

In [20]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Оценка качества на тестовой выборке
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"Точность на тестовой выборке: {test_accuracy:.4f}")

# Подробный отчет о классификации
print("\nОтчет о классификации:")
print(classification_report(y_test, y_test_pred))

# Количество правильных и неправильных предсказаний
correct_predictions = (y_test == y_test_pred).sum()
total_predictions = len(y_test)
print(f"\nПравильных предсказаний: {correct_predictions} из {total_predictions}")
print(f"Неправильных предсказаний: {total_predictions - correct_predictions}")

# Сравнение с базовым качеством
improvement = test_accuracy - 0.2168  # baseline accuracy
print(f"\nУлучшение по сравнению с бейзлайном: {improvement:.4f} ({improvement/0.2168*100:.1f}% относительного улучшения)")


Точность на тестовой выборке: 0.8320

Отчет о классификации:


/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                   precision    recall  f1-score   support

                        0.00      0.00      0.00        36
   69-я параллель       0.75      0.36      0.49       245
       Библиотека       0.00      0.00      0.00        16
           Бизнес       0.67      0.33      0.44      1453
      Бывший СССР       0.84      0.86      0.85     10767
              Дом       0.87      0.80      0.83      4338
         Из жизни       0.69      0.64      0.66      5441
   Интернет и СМИ       0.79      0.74      0.77      8988
             Крым       0.49      0.24      0.33       143
    Культпросвет        0.25      0.01      0.03        76
         Культура       0.88      0.89      0.89     10732
          Легпром       0.00      0.00      0.00        18
              Мир       0.82      0.86      0.84     27609
  Наука и техника       0.84      0.85      0.85     10502
      Путешествия       0.81      0.62      0.70      1281
           Россия       0.81      0.84      0.82     32

/Users/bitcoin/ITMO/ITMO-DL-NLP/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
# Анализ ошибок по классам
import matplotlib.pyplot as plt
import seaborn as sns

# Получим уникальные классы и их количество
unique_classes = y_test.unique()
print(f"Всего классов в тестовой выборке: {len(unique_classes)}")

# Посмотрим на распределение ошибок по классам
class_accuracy = {}
for cls in unique_classes:
    mask = y_test == cls
    if mask.sum() > 0:
        cls_accuracy = accuracy_score(y_test[mask], y_test_pred[mask])
        class_accuracy[cls] = cls_accuracy

# Топ-5 лучших и худших классов по точности
sorted_accuracy = sorted(class_accuracy.items(), key=lambda x: x[1])
print("\nТоп-5 худших классов по точности:")
for cls, acc in sorted_accuracy[:5]:
    count = (y_test == cls).sum()
    print(f"{cls}: {acc:.4f} ({count} образцов)")

print("\nТоп-5 лучших классов по точности:")
for cls, acc in sorted_accuracy[-5:]:
    count = (y_test == cls).sum()
    print(f"{cls}: {acc:.4f} ({count} образцов)")


Всего классов в тестовой выборке: 21

Топ-5 худших классов по точности:
: 0.0000 (36 образцов)
Легпром: 0.0000 (18 образцов)
Библиотека: 0.0000 (16 образцов)
ЧМ-2014: 0.0000 (1 образцов)
Культпросвет : 0.0132 (76 образцов)

Топ-5 лучших классов по точности:
Мир: 0.8561 (27609 образцов)
Бывший СССР: 0.8572 (10767 образцов)
Экономика: 0.8751 (15711 образцов)
Культура: 0.8928 (10732 образцов)
Спорт: 0.9704 (12761 образцов)


## Выводы по оценке качества лучшего пайплайна

**Лучший пайплайн:** TfidfVectorizer + LogisticRegression с оптимизированными гиперпараметрами

**Основные результаты:**
- Точность на тестовой выборке показывает стабильное качество модели
- Значительное улучшение по сравнению с базовым бейзлайном (dummy classifier)
- Модель показывает хорошую обобщающую способность на отложенных данных

**Характеристики модели:**
- Использует TF-IDF векторизацию для преобразования текста в числовые признаки
- Логистическая регрессия как классификатор с оптимизированными гиперпараметрами
- Предобработка включает лемматизацию, удаление стоп-слов и очистку текста
